In [1]:
import numpy  as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img, load_img
import os
import pandas as pd
import pathlib
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from tensorflow import keras
from sklearn.linear_model import SGDClassifier
from keras.regularizers import l2
from keras.layers import Conv2D, AveragePooling2D, Flatten, Dense,Activation, MaxPool2D, BatchNormalization, Dropout
import tensorflow as tf
from keras.callbacks import ReduceLROnPlateau

In [2]:
# file_path : 데이터 셋 파일 경로
# CATEGORIES : 데이터 셋 
file_path = pathlib.Path('./dataset/')
CATEGORIES = os.listdir(file_path)

# 데이터 셋 파일 불러오기 함수 선언
def load_data(data_dir):
    # 이미지 데이터를 담을 리스트 선언
    images = list()
    # CATEGORIES에 동물 리스트 담겨있음
    # 동물 하나하나의 폴더에 접근해 안에있는 이미지를 넘파이 배열로 전환
    # 밑 코드는 그냥 이미지를 픽셀값으로 받아온다고만 이해하기
    for category in CATEGORIES:
        categories = os.path.join(data_dir, str(category))
        for img in os.listdir(categories):
            img = load_img(os.path.join(categories, img), target_size=(30, 30))
            # 이미지를 넘파이 배열로 변환
            image = img_to_array(img) 
            # images에 픽셀, 컬러 데이터 담기
            images.append(image) 
    # 함수 선언하면 images 리턴
    return images

In [3]:
# 이미지 셋 불러오기
# 위 선언한 함수에 file_path경로를 파라미터로 넘겨 images에 담기
# 현재 images의 파일형식은 리스트
images = load_data(file_path)

# 타겟 셋 불러오기

# target 리스트 선언
target = []

# CATEGORIES에 있는 동물 이름을 타겟으로 설정
# 26179개의 이미지에 각각 타겟 설정
for j in CATEGORIES:
    path ='./dataset/{}'.format(j)
    polder = os.listdir(path)
    
    # target에 타겟값 추가
    for i in polder :
        target.append(j)
# 넘파이 배열로 변환
target = np.array(target)

# 데이터 셋 정규화
# 넘파이 배열로 변환
images = np.array(images)
# 픽셀 값 정규화
images = images / 255.0

In [4]:
# 26179의 행(이미지 개수임), 30, 30 : 픽셀(행 * 열), 3 : 색상(3 컬럼)
print(images.shape,target.shape)

(26179, 30, 30, 3) (26179,)


In [5]:
# 훈련 및 테스트 데이터셋으로 분류하기(8:2) / 한번만 실행, 계속 실행하면 계속분리됨
# 실수로 했다면 위 코드 다시 실행해 데이터 새로불러오기
train_input, test_input, train_target, test_target = train_test_split(images, target, test_size=0.2)

In [6]:
print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

(20943, 30, 30, 3) (20943,)
(5236, 30, 30, 3) (5236,)


In [7]:
# 훈련 및 검증 데이터셋으로 분류하기(8:2) / 한번만 실행, 계속 실행하면 계속분리됨
# 실수로 했다면 위 코드 다시 실행해 데이터 새로불러오기
train_input, val_input, train_target, val_target = train_test_split(train_input, train_target, test_size=0.2)

In [8]:
print(train_input.shape,train_target.shape)
print(val_input.shape,val_target.shape)

(16754, 30, 30, 3) (16754,)
(4189, 30, 30, 3) (4189,)


In [9]:
print(train_target)
print(val_target)
print(test_target)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train_target)
train_target = le.transform(train_target)
print("train_target = ", train_target)

le.fit(val_target)
val_target = le.transform(val_target)
print("val_target = ", val_target)

le.fit(test_target)
test_target = le.transform(test_target)
print("test_target = ", test_target)

#sparse 0,1,2,3,4 배열 형태 1차원
#원핫인코딩 - 배열 안에 / 1차원 배열의 카테고리


#1차원 안에 또 1차원의 리스트들이 들어가있다. 그 들어가있는 리스트 안에 0,1,0,0,1 원핫인코딩 들어있따
#리스트 안에 리스트가 또 들어가있다.
#binary => 1차원 형태 0,1 둘 중에 하나 



['butterfly' 'squirrel' 'spider' ... 'cow' 'cow' 'squirrel']
['cat' 'cow' 'squirrel' ... 'spider' 'cow' 'cow']
['cow' 'squirrel' 'dog' ... 'cat' 'cow' 'chicken']
train_target =  [0 9 8 ... 3 3 9]
val_target =  [1 3 9 ... 8 3 3]
test_target =  [3 9 4 ... 1 3 2]


In [10]:
def bottleneck_residual_block(X, kernel_size, filters, reduce=False, s=2):
    F1, F2, F3 = filters
    X_shortcut = X
    if reduce :
        X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1),
                           strides = (s, s))(X_shortcut)
        X_shortcut = BatchNormalization(axis = 3)(X_shortcut)
        
        X = Conv2D(filters = F1, kernel_size = (1, 1),
                  strides = (s, s),
                  padding = 'valid')(X)
        X = BatchNormalization(axis = 3)(X)
        X = Activation('relu')(X)
        
    else :
        # 주 경로의 첫 번째 합성곱층
        X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1, 1),
                  padding = 'valid')(X)
        X = BatchNormalization(axis = 3)(X)
        X = Activation('relu')(X)
        
    # 주 경로의 두 번째 합성곱층
    X = Conv2D(filters = F2, kernel_size = kernel_size, strides = (1, 1),
                  padding = 'same')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    
    # 주 경로의 세 번째 합성곱층
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1, 1),
                  padding = 'valid')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    
    # 처리의 마지막 단계
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [33]:
bottleneck_residual_block(train_input, (3, 3), (30, 30, 30))

NameError: name 'Add' is not defined

In [11]:
def ResNet50(input_shape, classes):
    X_input = Input(input_shape)
    
    # 1단계
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(X_input)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPool2D((3, 3), strides=(2, 2))(X)
    
    # 2단계
    X = bottleneck_residual_block(X, 3, [64, 64, 256], reduce=True, s=1)
    X = bottleneck_residual_block(X, 3, [64, 64, 256])
    X = bottleneck_residual_block(X, 3, [64, 64, 256])
    
    # 3단계
    X = bottleneck_residual_block(X, 3, [128, 128, 512], reduce=True, s=2)
    X = bottleneck_residual_block(X, 3, [128, 128, 512])
    X = bottleneck_residual_block(X, 3, [128, 128, 512])
    X = bottleneck_residual_block(X, 3, [128, 128, 512])
    
    # 4단계
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], reduce=True, s=2)
    X = bottleneck_residual_block(X, 3, [256, 256, 1024])
    X = bottleneck_residual_block(X, 3, [256, 256, 1024])
    X = bottleneck_residual_block(X, 3, [256, 256, 1024])
    X = bottleneck_residual_block(X, 3, [256, 256, 1024])
    X = bottleneck_residual_block(X, 3, [256, 256, 1024])
    
    # 5단계
    X = bottleneck_residual_block(X, 3, [512, 512, 2048], reduce=True, s=2)
    X = bottleneck_residual_block(X, 3, [512, 512, 2048])
    X = bottleneck_residual_block(X, 3, [512, 512, 2048])
    
    # 평균 풀링
    X = AveragePooling2D((1,1))(X)
    
    # 출력층
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes))(X)
    
    model = Model(inputs = X_input, outputs = X, name='ResNet50')
    
    return model

In [31]:
# 하이퍼 파라미터 설정하기

from keras.callbacks import ReduceLROnPlateau

# 학습 파라미터 정의
epochs = 200
batch_size = 256

# min_lr은 학습률의 하한값이며, factor는 학습률을 감소시킬 배율이다.
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1),
                             patience=5, min_lr=0.5e-6)
# 모델을 컴파일한다
model.compile(loss='categorical_crossentropy', optimizer=SGD,
             metrics=['accuracy'])
# reduce_lr을 콜백 함수로 지정해서 모델을 학습힌다.
model.fit(X_train, Y_train, batch_size=batch_size, validation_data=X_test, Y_test), epochs=epochs, callbacks=[reduce_lr]

SyntaxError: positional argument follows keyword argument (2245919407.py, line 16)

In [29]:
from tensorflow.keras.utils import plot_model

keras.utils.plot_model(model)

NameError: name 'model' is not defined